In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import pandas as pd
from sklearn.model_selection import train_test_split

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [5]:
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab190912.sh

Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 91, done.
remote: Counting objects: 100% (91/91), done.
remote: Compressing objects: 100% (85/85), done.
remote: Total 91 (delta 43), reused 22 (delta 6), pack-reused 0
Unpacking objects: 100% (91/91), done.
/content/Mecab-ko-for-Google-Colab
Installing konlpy.....
     |████████████████████████████████| 19.4MB 1.2MB/s 
     |████████████████████████████████| 450kB 46.3MB/s 
     |████████████████████████████████| 92kB 11.0MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2021-07-06 12:51:56--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 104.192.141.1, 2406:da00:ff00::6b17:d1f5, 2406:da00:ff00::22c2:513, ...
Con

In [6]:
from torchtext.legacy import data
from konlpy.tag import Mecab

In [7]:
cd /content/drive/MyDrive/torch_example/

/content/drive/MyDrive/torch_example


In [8]:
tokenizer = Mecab()

In [9]:
train = pd.read_csv("./data/nsmc/ratings_train.txt", sep='\t')
test = pd.read_csv("./data/nsmc/ratings_test.txt", sep='\t')

train = train.drop(columns=['id'])
test = test.drop(columns=['id'])

print(train.shape)
print(test.shape)

(150000, 2)
(50000, 2)


In [10]:
train_data = train.dropna() #말뭉치에서 nan 값을 제거함
test_data = test.dropna()

print(train_data.shape)
print(test_data.shape)

(149995, 2)
(49997, 2)


In [11]:
train_data, valid_data = train_test_split(train_data, test_size=0.3, random_state=32)

In [12]:
print(len(train_data))
print(len(valid_data))

104996
44999


In [13]:
TEXT = data.Field(sequential=True, use_vocab=True, tokenize=tokenizer.morphs, lower=False, batch_first=True, fix_length=20)
LABEL = data.LabelField(dtype = torch.float) 

In [14]:
def convert_dataset(input_data, text, label):
    list_of_example = [data.Example.fromlist(row.tolist(), fields=[('text', text), ('label', label)])  for _, row in input_data.iterrows()]
    dataset = data.Dataset(examples=list_of_example, fields=[('text', text), ('label', label)])
    return dataset

In [15]:
train_data = convert_dataset(train_data,TEXT,LABEL)
valid_data = convert_dataset(valid_data, TEXT, LABEL)

In [16]:
TEXT.build_vocab(train_data, max_size=10000)
LABEL.build_vocab(train_data)

In [17]:
print('단어 집합의 크기 : {}'.format(len(TEXT.vocab)))
print('label 의 크기 : {}'.format(len(LABEL.vocab)))

단어 집합의 크기 : 10002
label 의 크기 : 2


10000개 단어 + \<unk> + \<pad> 토큰이 들어감


In [18]:
print(LABEL.vocab.stoi) #라벨 값이 

defaultdict(None, {0: 0, 1: 1})


In [19]:
test_data = convert_dataset(test_data, TEXT, LABEL)

In [20]:
batch_size = 5
train_iter, valid_iter, test_iter = data.Iterator.splits((train_data, valid_data, test_data), batch_size=batch_size, sort=False, device=device)

In [21]:
print('훈련 데이터의 미니 배치 수 : {}'.format(len(train_iter)))
print('평가 데이터의 미니 배치 수 : {}'.format(len(valid_iter)))
print('테스트 데이터의 미니 배치 수 : {}'.format(len(test_iter)))

훈련 데이터의 미니 배치 수 : 21000
평가 데이터의 미니 배치 수 : 9000
테스트 데이터의 미니 배치 수 : 10000


text를 cnn으로 돌릴때, conv2d와 conv1d로 나눠서 돌릴 수 있다.

CNN을 1d 로 볼 때, input_chanel은 embed dim이다. 

In [36]:
class CNN1D(nn.Module):
  def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, 
                 dropout, pad_idx):
    super(CNN1D, self).__init__()
    self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_idx)

    self.conv = nn.ModuleList([ nn.Conv1d(in_channels=embedding_dim, out_channels=n_filters, kernel_size=filter, padding='same')\
                               for filter in filter_sizes])

    self.fc = nn.Linear( n_filters * len(filter_sizes), output_dim)
    self.dropout = nn.Dropout(dropout)

  def forward(self, x):
    embedded = self.embedding(x)
    #embedded : batch x seq len x embed dim

    embedded = embedded.permute(0,2,1)
    #embedded : batch x embed dim x seq len

    conv_out = [F.relu(conv(embedded)) for conv in self.conv] 
    #conv_out : batch x feature map x (sent len - filter_size[n] + 1)
    # == batch * feature map * seq len

    #pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conv_out]
    # pooled : batch x feature map

    output = self.fc(self.dropout(torch.cat(pooled, dim =1)))

    return output

In [23]:
class Self_attn(nn.Module):
  def __init__(self, in_dim):
    super(Self_attn, self).__init__()
    
    self.

  def forward(self, x):
    '''
    inputs :
      x : input feature map (batch x seq len x  )
    '''
    

In [24]:
def binary_accuracy(prediction, target):
  '''
from https://github.com/bentrevett/pytorch-sentiment-analysis/blob/master/1%20-%20Simple%20Sentiment%20Analysis.ipynb
  '''
  # round predictions to the closest integer (0 or 1)
  rounded_preds = torch.round(torch.sigmoid(prediction))
  
  #convert into float for division
  correct = (rounded_preds == target).float()

  # rounded_preds = [ 1   0   0   1   1   1   0   1   1   1]
  # targets       = [ 1   0   1   1   1   1   0   1   1   0]
  # correct       = [1.0 1.0 0.0 1.0 1.0 1.0 1.0 1.0 1.0 0.0]
  acc = correct.sum() / len(correct)
  return acc

In [25]:
def train(model, train_iter):
  model.train()

  epoch_loss, epoch_acc = 0, 0
  for batch in train_iter:
    optimizer.zero_grad()
    x, y = batch.text.to(device), batch.label.to(device)
    y_hat = model(x).squeeze(1)

    loss = criterion(y_hat, y)
    acc = binary_accuracy(y_hat, y)

    loss.backward()
    optimizer.step()

    epoch_loss += loss.item()
    epoch_acc += acc.item()

  return epoch_loss / len(train_iter), epoch_acc / len(train_iter)


In [26]:
def evaluate(model, valid_iter):
  model.eval()
  with torch.no_grad():
    eval_loss, eval_acc = 0, 0
    for batch in valid_iter:
      x,y = batch.text.to(device), batch.label.to(device)
      y_hat = model(x).squeeze(1)
      
      loss = criterion(y_hat, y)
      acc = binary_accuracy(y_hat, y)

      eval_loss += loss.item()
      eval_acc += acc.item()
      
  return eval_loss / len(valid_iter), eval_acc / len(valid_iter)

In [27]:
def inference(model, test_iter):
  model.eval()
  with torch.no_grad():
    test_loss, test_acc = 0, 0
    for batch in test_iter:
      x, y = batch.text.to(device), batch.label.to(device)
      y_hat = model(x).squeeze(1)

      loss = criterion(y_hat, y)
      acc = binary_accuracy(y_hat, y)
      
      test_loss += loss.item()
      test_acc += acc.item()

  return test_loss / len(test_iter), test_acc / len(test_iter)

In [37]:
#  vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, dropout, pad_idx
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = CNN1D(len(TEXT.vocab), 128, 100, [3,4,5], 1, 0.2, PAD_IDX)
model.to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.BCEWithLogitsLoss()
criterion.to(device)

BCEWithLogitsLoss()

In [38]:
best_val_loss = float('inf')
for _epoch in range(1,2): # 5 epoch
  train_loss, train_acc = train(model, train_iter)
  valid_loss, valid_acc = evaluate(model, valid_iter)
  print("[Epoch: %d] train loss : %5.2f | train accuracy : %5.2f" % (_epoch, train_loss, train_acc))
  print("[Epoch: %d] val loss : %5.2f | val accuracy : %5.2f" % (_epoch, valid_loss, valid_acc))

    # 검증 오차가 가장 적은 최적의 모델을 저장
  if valid_loss < best_val_loss:
    best_val_loss = valid_loss

enbed torch.Size([5, 128, 20])
torch.Size([5, 100, 20])
enbed torch.Size([5, 128, 20])
torch.Size([5, 100, 20])
enbed torch.Size([5, 128, 20])
torch.Size([5, 100, 20])
enbed torch.Size([5, 128, 20])
torch.Size([5, 100, 20])
enbed torch.Size([5, 128, 20])
torch.Size([5, 100, 20])
enbed torch.Size([5, 128, 20])
torch.Size([5, 100, 20])
enbed torch.Size([5, 128, 20])
torch.Size([5, 100, 20])
enbed torch.Size([5, 128, 20])
torch.Size([5, 100, 20])
enbed torch.Size([5, 128, 20])
torch.Size([5, 100, 20])
enbed torch.Size([5, 128, 20])
torch.Size([5, 100, 20])
enbed torch.Size([5, 128, 20])
torch.Size([5, 100, 20])
enbed torch.Size([5, 128, 20])
torch.Size([5, 100, 20])
enbed torch.Size([5, 128, 20])
torch.Size([5, 100, 20])
enbed torch.Size([5, 128, 20])
torch.Size([5, 100, 20])
enbed torch.Size([5, 128, 20])
torch.Size([5, 100, 20])
enbed torch.Size([5, 128, 20])
torch.Size([5, 100, 20])
enbed torch.Size([5, 128, 20])
torch.Size([5, 100, 20])
enbed torch.Size([5, 128, 20])
torch.Size([5, 10

KeyboardInterrupt: ignored

In [ ]:
test_loss, test_acc = inference(model, test_iter)
print('Test Loss: %5.2f | Test Acc: %5.2f '%(test_loss, test_acc*100))